Import dependencies

In [ ]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import hopsworks

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier, plot_importance
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from imblearn.over_sampling import SMOTE, RandomOverSampler
from collections import Counter
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold, RandomizedSearchCV

In [ ]:
project = hopsworks.login()
fs = project.get_feature_store()

Fetch the data

In [ ]:
wine_quality = pd.read_csv("Wine/winequality/winequality-white.csv", sep=";")
df_features = wine_quality

In [ ]:
def visualize_dataset(data, columns):
    fig, axis = plt.subplots(3, 4)
    c = 0
    for i in range(3):
        for j in range(4):
            if c == 11:
                break
            axis[i,j].title.set_text(columns[c])
            data[columns[c]].hist(bins = 20, ax = axis[i,j])
            c += 1
    axis[2,3].title.set_text("quality")
    data["quality"].hist(bins = 20, ax = axis[2,3])
    plt.show()

columns = wine_quality.columns
visualize_dataset(wine_quality, columns)

In [ ]:
df_features = wine_quality

drop_col = ["fixed_acidity",
           "citric_acid",
           "total_sulfur_dioxide",
           "density",
           "pH",
           "sulphates",
           ]

df_features.drop(columns = drop_col, inplace = True)
df_features.drop_duplicates(inplace = True)
X, y = df_features.iloc[:,:-1], df_features.iloc[:,-1:]


In [ ]:
columns = [
           "volatile_acidity",
           "residual_sugar",
           "chlorides",
           "free_sulfur_dioxide",
           "alcohol",
           "quality"
           ]


v = y.values.squeeze()
v1 = np.select([v<6,v==6, v>6],[0,1,2])[:,None]
y = pd.DataFrame(v1, columns=["quality"])

X_val = X.values
y_val = y.values
df_data = np.concatenate((X_val, y_val), axis = 1)

df = pd.DataFrame(df_data, columns=columns)

grouping = df.groupby('quality')


In [ ]:
wine_fg = fs.get_or_create_feature_group(
    name="wine",
    version=1,
    primary_key=columns, 
    description="Dataset containing properties of different white wines and their respective qualities")
wine_fg.insert(df)

for key in grouping.groups.keys():
    fg = fs.get_or_create_feature_group(
    name="wine_subset_" + str(int(key)),
    version=1,
    primary_key=columns, 
    description="Wine that belong to quality group: " +  str(key),
    parents=[wine_fg])
    v1 = grouping.get_group(key)
    fg.insert(v1)